# Dynamic Programming

In [1]:
import numpy as np
import gym

from IPython.display import clear_output
from time import sleep

#these helpers are located in a local folder
from utils.helper import create_random_policy
from utils.helper import print_policy, print_state_value_func

## Policy Iteration

In [2]:
# policy evaluation
def policy_evaluation(pi, env, gamma=1.0, delta=1e-10):
    '''
    Calculate the value function of a given policy pi
    
    Args: 
        pi:      policy to be evaluated (returns an action given a state)
        env:     openai gym environment
        gamma:   discount factor
        delta:   threshhold value to interrupt the policy evaluation
    
    Returns: 
        State value function V
    '''
    # P represents environment dynamics, including transition probabilities
    # P[state][action] = list of: probability of transition into next state, next state, reward, final state flag
    P = env.env.P 
    nS = env.observation_space.n
    
    # initialize a value function with 0
    V = np.zeros(nS, dtype=np.float64)
    V_old = V.copy()    
    while True:
        for state in range(nS):
            action = pi(state)
            v = 0
            for pr, next_state, reward, done in P[state][action]:
                v += pr * (reward + gamma * V[next_state] * (not done))
            V[state] = v
        
        max_diff = np.max(np.abs(V_old - V))
        if max_diff < delta:
            break
        
        V_old = V.copy()
    return V

In [3]:
# policy improvement
def policy_improvement(env, V, gamma=1.0):
    '''
    Improve an existing strategy by acting greedily
    
    Args: 
        env:     openai gym environment
        V:       state value function of a given policy
        gamma:   discount factor
    
    Returns: 
        pi:      a policy acting greedily using the value function of the current policy
    
    '''
    nS = env.observation_space.n
    nA = env.action_space.n
    P = env.env.P
    Q = np.zeros((nS, nA))
    
    for state in range(nS):
        for action in range(nA):
            for pr, next_state, reward, done in P[state][action]:
                Q[state][action] += pr * (reward + gamma * V[next_state] * (not done))
    
    
    greedy_strategy = {s:a for s, a in enumerate(np.argmax(Q, axis=1))}
    
    def pi(s):
        return greedy_strategy[s]
    
    return pi

In [4]:
# policy iteration
def policy_iteration(pi, env, gamma=1, delta=1e-10):
    '''
    Finds an optimal policy 
    
    Args: 
        pi:      starting policy
        env:     openai gym environment
        gamma:   discount factor
        delta:   threshhold value to interrupt the policy evaluation
    
    Returns: 
        Optimal policy and value function
    '''
    
    
    old_strategy = {s: pi(s) for s in range(env.observation_space.n)}
    
    while True:
        V = policy_evaluation(pi, env, gamma, delta)
        new_pi = policy_improvement(env, V, gamma)
        new_strategy = {s: new_pi(s) for s in range(env.observation_space.n)}
        
        if old_strategy == new_strategy:
            break
            
        old_strategy = new_strategy
        pi = new_pi
        
    return new_pi, policy_evaluation(new_pi, env, gamma, delta)

## Value Iteration

In [5]:
def value_iteration(env, gamma=1, delta=1e-10):
    '''
    Finds an optimal policy
    
    Args: 
        env:     openai gym environment
        gamma:   discount factor
        delta:   threshhold value to interrupt the policy evaluation
    
    Returns: 
        Optimal policy and value function
    ''' 
    nS = env.observation_space.n
    nA = env.action_space.n
    V = np.zeros(nS, dtype=np.float64)
    P = env.env.P
    
    while True:
        V_old = V.copy()
        Q = np.zeros(shape=(nS, nA), dtype=np.float64)
        for state in range(nS):
            for action in range(nA):
                for pr, next_state, reward, done in P[state][action]:
                    Q[state][action] += pr * (reward + gamma * V[next_state] * (not done))
        V = np.max(Q, axis=1)
        max_diff = np.max(np.abs(V_old - V))
        if max_diff < delta:
            break

    strategy = {s: a for s, a in enumerate(np.argmax(Q, axis=1))}
    def pi(s):
        return strategy[s]
    
    return pi, V

## Frozen Lake

In [26]:
env = gym.make('FrozenLake-v0')

In [27]:
env.render()


SFFF
FHFH
FFFH
HFFG


In [28]:
random_policy = create_random_policy(env.observation_space.n, env.action_space.n)

In [29]:
V = policy_evaluation(random_policy, env)

In [30]:
print_policy(random_policy, 16, 4, name='Frozen Lake random strategy')



Frozen Lake random strategy


         ←          ←          →          ↓
         ↓          ■          ←          ■
         ↑          ←          ←          ■
         ■          ↓          ←          ■


The random policy performs rather poorly.

In [31]:
print_state_value_func(V, 4, 'Frozen Lake random strategy value function')



Frozen Lake random strategy value function


0.00000 0.00000 0.00000 0.00000
0.00000 0.00000 0.00000 0.00000
0.00000 0.00000 0.00000 0.00000
0.00000 0.00000 0.00000 0.00000


After a single step of policy improvement 

In [32]:
new_policy = policy_improvement(env, V)

In [33]:
V = policy_evaluation(new_policy, env)

In [34]:
print_policy(new_policy, 16, 4, name='Improved policy')



Improved policy


         ←          ←          ←          ←
         ←          ■          ←          ■
         ←          ←          ←          ■
         ■          ←          ↓          ■


In [35]:
print_state_value_func(V, 4, 'Value function: improved policy')



Value function: improved policy


0.00000 0.00000 0.03846 0.01923
0.00000 0.00000 0.07692 0.00000
0.00000 0.00000 0.19231 0.00000
0.00000 0.00000 0.50000 0.00000


In [36]:
pi, V = policy_iteration(random_policy, env, gamma=0.99)

In [37]:
print_policy(pi, 16, 4, name='Optimal policy by policy iteration')



Optimal policy by policy iteration


         ←          ↑          ↑          ↑
         ←          ■          ←          ■
         ↑          ↓          ←          ■
         ■          →          ↓          ■


In [38]:
print_state_value_func(V, 4, name='Optimal value function by policy iteration')



Optimal value function by policy iteration


0.54203 0.49880 0.47070 0.45685
0.55845 0.00000 0.35835 0.00000
0.59180 0.64308 0.61521 0.00000
0.00000 0.74172 0.86284 0.00000


In [39]:
pi, V = value_iteration(env, gamma=0.99)

In [40]:
print_policy(pi, 16, 4, name='Optimal policy by value iteration')



Optimal policy by value iteration


         ←          ↑          ↑          ↑
         ←          ■          ←          ■
         ↑          ↓          ←          ■
         ■          →          ↓          ■


In [41]:
print_state_value_func(V, 4, name="Optimal value function by value iteration")



Optimal value function by value iteration


0.54203 0.49880 0.47070 0.45685
0.55845 0.00000 0.35835 0.00000
0.59180 0.64308 0.61521 0.00000
0.00000 0.74172 0.86284 0.00000


In [46]:
#Play an episode of frozen lake
obs, done = env.reset(), False
env.render()
while not done:
    sleep(0.5)
    clear_output(wait=True)
    action = pi(obs)
    next_obs, reward, done, _ = env.step(action)
    obs = next_obs
    env.render()

  (Down)
SFFF
FHFH
FFFH
HFFG


## Taxi

In [47]:
env = gym.make('Taxi-v3')
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [48]:
pi, V = value_iteration(env, gamma=0.99)

In [49]:
# Play an episode of taxi
obs, done = env.reset(), False
env.render()
while not done:
    sleep(0.5)
    clear_output(wait=True)
    action = pi(obs)
    next_obs, reward, done, _ = env.step(action)
    obs = next_obs
    env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)
